In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.4
# UPPPPPPPPPPPPPPPPPPPPPPPPPPPPDATED!!!!!!!!!!!!!
from project_lib import Project
project = Project(project_id='be25fa40-e354-4dbf-96a3-5e9a80976251', project_access_token='')


<h1 style="float: left; margin-left: 1em; margin-top: 0.6em;">Compare the AutoAI model with the reference one and select the winning</h1>

This notebook contains steps and code to demonstrate how to compare a model trained using AutoAI experiment with a reference one, trained using another notebook code.
Both notebooks are triggered within execution of a Watson Pipeline, that passes model paths and evaluation results as environment variables.

Some familiarity with Python is helpful. This notebook uses **Python 3.10** with the <a href="https://pypi.python.org/pypi/ibm-watson-pipelines" target="_blank" rel="noopener no referrer"><code>ibm-watson-pipelines</code></a> package.

### Contents

- [Setup](#setup)
- [Model selection](#select)
- [Storing notebook results](#results)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use <a href="https://cloud.ibm.com/docs/cli/index.html" target="_blank" rel="noopener no referrer">IBM Cloud CLI</a> to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


**Tip**: Your `Cloud API key` can be generated by going to the <a href="https://cloud.ibm.com/iam#/users" target="_blank" rel="noopener no referrer"><strong>Users</strong> section of the Cloud console</a>. From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the <a href="https://cloud.ibm.com/apidocs/machine-learning" target="_blank" rel="noopener no referrer"><strong>Endpoint URLs</strong> section of the Watson Machine Learning docs</a>. You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the <a href="https://cloud.ibm.com/iam/serviceids" target="_blank" rel="noopener no referrer"><strong>Service IDs</strong> section of the Cloud Console</a>. From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` in the following cell.

In [2]:
api_key = '1cgCoLRH-5Kp4Y67XMm9FjdBwGxR-3prvleXZ9qK4pt0'


<strong style="color: crimson;">Beta program notice: </strong>Only **`us-south`** location is supported in the beta program.

In [3]:
cpdaas_url = 'https://cpd-cpd-instance.apps.daffy-hrb7pxfs.cloud.techzone.ibm.com'


## Package installation

In [4]:
!pip install -U ibm-watson-machine-learning | tail -n 1
!pip install -U ibm-watson-pipelines | tail -n 1

In [5]:
import os
import numpy as np

from ibm_watson_machine_learning import APIClient
from ibm_watson_pipelines import WatsonPipelines
from ibm_watson_pipelines.cpd_paths import CpdScope, CpdPath

## Notebook parameters

This notebook is run from the Watson Pipeline that passes following environment variables:

- `autoai_model` - the AutoAI model (as CPD Path, e.g. 'cpd:///spaces/b5ad9f7e-eec3-49ad-a8f6-86354ff29134/assets/b6046bcd-7c20-4fc8-bb70-9ed0944d185b')
- `autoai_model_metric` - the AutoAI model evaluation result (as float, e.g. '0.7')
- `reference_model` - the reference model (as CPD Path, e.g. 'cpd:///spaces/b5ad9f7e-eec3-49ad-a8f6-86354ff29134/assets/b6046bcd-7c20-4fc8-bb70-9ed0944d185b')
- `reference_model_metric` - the reference model evaluation result (as float, e.g. '0.67')


In [7]:
#REFERENCE_MODEL_METRIC = float(0.79)
#AUTOAI_MODEL_METRIC = float(0.8148837209302325)
#AUTOAI_MODEL =    'cpd:///spaces/8a4bb896-8b3f-4bbd-b1e6-df80b7a7d3e6/assets/d13b5d68-1ad4-4ea5-85ba-34bcf5e3302e'
#REFERENCE_MODEL = 'cpd:///spaces/8a4bb896-8b3f-4bbd-b1e6-df80b7a7d3e6/assets/c9efe3d5-b1c7-4da8-9396-73e1933851ff'

AUTOAI_MODEL = os.getenv('autoai_model')
AUTOAI_MODEL_METRIC = float(os.getenv('autoai_model_metric'))
REFERENCE_MODEL = os.getenv('reference_model')
REFERENCE_MODEL_METRIC = float(os.getenv('reference_model_metric'))


REFERENCE_MODEL_ID = CpdPath.from_string(REFERENCE_MODEL).resource_id()
AUTOAI_MODEL_ID = CpdPath.from_string(AUTOAI_MODEL).resource_id()

print('AutoAI model:', AUTOAI_MODEL_ID)
print('AutoAI model metric:', AUTOAI_MODEL_METRIC)
print('Reference model:', REFERENCE_MODEL_ID)
print('Reference model metric:', REFERENCE_MODEL_METRIC)

AutoAI model: d13b5d68-1ad4-4ea5-85ba-34bcf5e3302e
AutoAI model metric: 0.8148837209302325
Reference model: c9efe3d5-b1c7-4da8-9396-73e1933851ff
Reference model metric: 0.79


## Connect to Watson Machine Learning

Select the deployment space where the reference model, pointed in `reference_model` environment variable, is stored.

In [8]:
pipelines_client = WatsonPipelines.new_instance()
print('pipelines_client:', pipelines_client)

# Extract space ID from reference model CPD Path passed from the Watson Pipeline as an environment variable
space_scope = CpdPath.from_string(REFERENCE_MODEL)
print('space_scope', space_scope)
space_id = space_scope.scope_id()
print ('space_id:', space_id)
print('space_scope.scope():', space_scope.scope())

wml_credentials = {  "version": "4.7",
    "url": "https://cpd-cpd-instance.apps.daffy-hrb7pxfs.cloud.techzone.ibm.com/",
    "username": "ckroustalis",
    "password": "predicta1!",
    "instance_id": "openshift"}
                   
                   
print('wml_credentials:', wml_credentials)              

pipelines_client: <ibm_watson_pipelines.client.WatsonPipelines object at 0x7f097ea35000>
space_scope cpd:///spaces/8a4bb896-8b3f-4bbd-b1e6-df80b7a7d3e6/assets/c9efe3d5-b1c7-4da8-9396-73e1933851ff
space_id: 8a4bb896-8b3f-4bbd-b1e6-df80b7a7d3e6
space_scope.scope(): cpd:///spaces/8a4bb896-8b3f-4bbd-b1e6-df80b7a7d3e6
wml_credentials: {'version': '4.7', 'url': 'https://cpd-cpd-instance.apps.daffy-hrb7pxfs.cloud.techzone.ibm.com/', 'username': 'ckroustalis', 'password': 'predicta1!', 'instance_id': 'openshift'}


In [9]:
wml_client = APIClient(wml_credentials)

wml_client.set.default_space(space_id)

'SUCCESS'

<a id="selection"></a>
## 2. Model selection

The best model is selected based on an evaluation metric value.

In [10]:
if AUTOAI_MODEL_METRIC > REFERENCE_MODEL_METRIC:
    print('Selected AutoAI model')
    selected_model_id = AUTOAI_MODEL_ID
    selected_model = AUTOAI_MODEL
else:
    print('Selected reference model')
    selected_model_id = REFERENCE_MODEL_ID
    selected_model = REFERENCE_MODEL

Selected AutoAI model


Mark selected model with `selected_model` tag.

In [11]:
not_selected_metadata = {
    wml_client.repository.ModelMetaNames.TAGS: []
}
for model in wml_client.repository.get_model_details()['resources']:
    model_id = wml_client.repository.get_model_id(model)
    wml_client.repository.update_model(model_id, not_selected_metadata)

selected_metadata = {
    wml_client.repository.ModelMetaNames.TAGS: ["selected_model"]
}
wml_client.repository.update_model(selected_model_id, selected_metadata)

print('Models tagged.')

Models tagged.


<a id="results"></a>
## 3. Save notebook results

This notebook is run from the Watson Pipeline that expects following resuklts:

- `selected_model` - the selected model CPD Path


In [12]:
results = {
  "selected_model": selected_model
}
print(results)

{'selected_model': 'cpd:///spaces/8a4bb896-8b3f-4bbd-b1e6-df80b7a7d3e6/assets/d13b5d68-1ad4-4ea5-85ba-34bcf5e3302e'}


Finally, store the results, which are consumed by a flow downstream nodes, using the Watson Pipelines client.

In [14]:
res = pipelines_client.store_results(results)

print('DONE')

Running outside of Watson Pipelines - storing results in the local filesystem for testing purposes...

  output paths:
    - "selected_model": .ibm_watson_pipelines/results/selected_model
DONE


### Authors

**Rafał Bigaj**, System Architect with long successful record of building and leading teams. Broad and practical knowledge in the area of cloud computing, machine learning and distributed systems development.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.